In [2]:
#colab cell
#from google.colab import drive
#drive.mount('/content/drive')
!unzip /content/drive/MyDrive/Colab\ Notebooks/Phase2.zip
!ls

Archive:  /content/drive/MyDrive/Colab Notebooks/Phase2.zip
   creating: DataSet2/
   creating: DataSet2/Without-Noise/
  inflating: DataSet2/Without-Noise/cast_def_0_0.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_2.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_22.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_25.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_33.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_79.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_91.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_93.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_94.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_100.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_102.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_127.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_138.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_146.jpeg  
  inflating: DataSet2/Without-Noise/cast_def_0_150.jpeg  
  inflating: DataSet2

In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
# Load image file paths and corresponding noise labels from labels.xlsx
labels_df = pd.read_excel('Labels.xlsx', names=['Original_Path', 'Noise_Type', 'Noisy_Path'])
target_size = 400

In [3]:
# Preprocess the data
image_paths = labels_df['Original_Path'].tolist()
noise_labels = labels_df['Noise_Type'].tolist()


In [4]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(image_paths, noise_labels, test_size=0.1, random_state=42)
# Map string labels to numerical values
label_mapping = {'Gaussian': 0, 'Salt': 1, 'Periodic': 2}

# Convert string labels to numerical labels
y_train_numerical = [label_mapping[label] for label in y_train]
y_val_numerical = [label_mapping[label] for label in y_val]


In [5]:
# Define a function to load and preprocess images
def load_and_preprocess_image(image_path):
    dataset_path = '/content/DataSet2/' + image_path
    img = cv2.imread(dataset_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (target_size, target_size))  # Resize the image to the desired input shape
    img = img / 255.0  # Normalize pixel values to the range [0, 1]
    return img
# Load and preprocess images
X_train_processed = np.array([load_and_preprocess_image(path) for path in X_train])
X_val_processed = np.array([load_and_preprocess_image(path) for path in X_val])


In [6]:
# Define the noise classifier model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(target_size, target_size, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Output layer with 3 classes for the types of noise
])


In [7]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [8]:
# Train the noise classifier model
model.fit(X_train_processed, np.array(y_train_numerical), epochs=10, validation_data=(X_val_processed, np.array(y_val_numerical)))


Epoch 1/10
36/36 [==============================] - 376s 10s/step - loss: 4.0863 - accuracy: 0.3316 - val_loss: 1.0989 - val_accuracy: 0.3280
Epoch 2/10
 2/36 [>.............................] - ETA: 5:51 - loss: 1.0971 - accuracy: 0.4688

KeyboardInterrupt: 

In [ ]:
# Evaluate the noise classifier model
y_pred = model.predict(X_val_processed)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_val_numerical, y_pred_classes)
conf_matrix = confusion_matrix(y_val_numerical, y_pred_classes)
classification_rep = classification_report(y_val_numerical, y_pred_classes)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)